# GAMES ANALYSIS PROJECT

# Setup

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# loading the datasets.
# i have already downloaded the datasets from the following links:

# https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data
games_data = pd.read_csv(r'C:\!Projects\games_analysis\Data\games.psd')
games = games_data.copy()

# https://www.kaggle.com/datasets/hibrahimag1/top-1000-twitch-streamers-data-may-2024
twitch_streamers = pd.read_csv(r'C:\!Projects\games_analysis\Data\Twitch_streamers_data.csv')
streamers = twitch_streamers.copy()

# https://www.kaggle.com/datasets/rankirsh/evolution-of-top-games-on-twitch
twitch_stat = pd.read_csv(r'C:\!Projects\games_analysis\Data\twitch_games_data.csv', encoding='latin1')
twitch = twitch_stat.copy()

# EDA

## Quick view of the datasets

In [ ]:
games.head()

In [ ]:
streamers.head()

In [ ]:
twitch.head()

## Missings

In [ ]:
games.info()

In [ ]:
games.isnull().sum()

In [ ]:
# i will drop columns with 50% more missings
missing_percentage = games.isnull().mean()*100
N = 50
columns_to_keep = missing_percentage[missing_percentage<=N].index
games = games[columns_to_keep]

In [ ]:
games.isnull().mean()*100

In [ ]:
games.loc[games.Name.isnull()]

In [ ]:
# i will drop the rows witn nan in the name column
print(games.shape)
games = games.dropna(subset=['Name'])
print(games.shape)

In [ ]:
games.isnull().sum()

In [ ]:
games.loc[games['About the game'].isnull()]

In [ ]:
# i see that most of this games are 'playtest' so i will add this description
games.loc[games.Name.str.contains('Playtest'),'About the game'] = "This is a playtest game is the process by which a game designer tests a new game for bugs and design flaws before releasing it to market."

In [ ]:
games.isnull().mean()

In [ ]:
games['About the game'].fillna('The Game does not have a description', inplace=True)

In [ ]:
games.isnull().sum()

In [ ]:
games.loc[games['Support email'].isnull()]

In [ ]:
# for others rows
games['Support email'] = games['Support email'].fillna('Unknown')
games['Developers'] = games['Developers'].fillna('Unknown')
games['Publishers'] = games['Publishers'].fillna('Unknown')

In [ ]:
# for other missing i just will fill with 'NA'
games.isnull().sum()

In [ ]:
games.info()

In [ ]:
games.fillna('Unknown', inplace=True)

In [ ]:
games.isnull().sum()

## Duplicated rows

In [ ]:
games.duplicated().sum()

## Other manipulations

In [ ]:
# changing date column type to dt
games['Release date'] = pd.to_datetime(games['Release date'], format='mixed')

In [ ]:
games['Release date'].describe()

In [ ]:
# i will remove rows where data year = 2025
games = games[games['Release date'].dt.year != 2025]

In [ ]:
# take only last 10 years
games = games[games['Release date'].dt.year >= 2013]

In [ ]:
games['Release date'].describe()

In [ ]:
games.info()

In [ ]:
games.describe().T

In [ ]:
count = games[games['Estimated owners'] == "0 - 0"]
count

In [ ]:
# i will drop this rows aswell
print(games.shape)
games = games[games['Estimated owners']!='0 - 0']
print(games.shape)

In [ ]:
games['Price'].describe()

In [ ]:
games.groupby('Price').size().reset_index(name='Count')

In [ ]:
print(games.shape)
games = games[games['Price'] < 900]
print(games.shape)

## Visualisations

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(games.corr(numeric_only=True), annot=False)

In [ ]:
pb = games['Publishers'].value_counts().head(10)

plt.figure(figsize=(10,6))
sns.barplot(x= pb.index, y= pb.values, palette='rocket')
plt.title("Top Publishers Based on Games quantity")
plt.xlabel("Publishers")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
metacrit = ~(games['Metacritic score']==0)

meta_filtered = games[metacrit]

meta_filtered.groupby('Name')[['Metacritic score']].mean().nlargest(10, 'Metacritic score')

In [ ]:
df1 = games.dropna(subset=['Tags'])
genres = df1['Genres'].value_counts().nlargest(10)

total_games = genres.sum()
percentages = [(count / total_games) * 100 for count in genres]

plt.figure(figsize=(12, 6))
barplot = sns.barplot(y=genres.index, x=genres.values, palette='muted', orient='h')

for i, (count, percentage) in enumerate(zip(genres, percentages)):
    barplot.text(genres.values[i] + 2, i, f'{percentage:.1f}%', ha='left', va='center', fontsize=10)

plt.xlabel("Total Estimated Owners", fontsize=12)
plt.ylabel("Second Tag", fontsize=12)
plt.title('Most PLayed Genres by Owners', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

# AI

In [ ]:
games.tail()

In [ ]:
# Using Gemini OpenAI
import google.generativeai as genai
import apikey

key = apikey.load("game_api")
genai.configure(api_key=key)

model = genai.GenerativeModel('gemini-1.5-flash')
# response = model.generate_content("Write a story about a magic backpack.")
# print(response.text)

# POWER BI

In [ ]:
PBI = games.copy()

In [ ]:
PBI.to_csv(r'C:\Users\kyana\OneDrive\Рабочий стол\DEMODAY\games.csv', index=False, encoding='utf-8')